In [4]:
# 修改checkpoint路径
ckpt_dir = '/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/openchat_3.5/with_solar_info/brave/data1-split=8:2-ratio=1.0/lr=1.2e-4-20240117-03:47:14'

import os
import sys
import json
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

dirs = [".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    get_model_tokenizer, get_template, inference
)
from swift.tuners import Swift
from custom import CustomModelType, CustomTemplateType
import evaluation

with open(f"{ckpt_dir}/sft_args.json", "r") as f:
    sft_args = json.load(f)
model_type, template_type = sft_args["model_type"], sft_args["template_type"]
model, tokenizer = get_model_tokenizer(model_type, model_kwargs={'device_map': 'auto'})
model = Swift.from_pretrained(model, ckpt_dir, inference_mode=True)
model.generation_config.max_new_tokens = 512
model.generation_config.do_sample = False

template = get_template(template_type, tokenizer)

get_response = lambda prompt : inference(model, template, prompt)


[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/openchat-3.5-0106",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.36.2",
  "use_cache": true,
  "vocab_size": 32002
}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: Mixed using with peft is not allowed now.

In [3]:
wrong_ans = evaluation.cal_metric_single_llm(sft_args, get_response, save=True)


9 / 2439
{0: 7, 2: 1, -2: 0, 'wrong': 0}
ACC: 1.0
F1: 1.0
Precision: 1.0
Recall: 1.0


Prompt:
According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-04-03
Content: A map shows that Madagascar is the only country without coronavirus cases.

PRIOR KNOWLEDGE:
Information 1:
Publication date: 2020-04-03
Title: FALSE: A map shows that Madagascar is the only country without ...
Content:
Home <strong>A</strong> <strong>map</strong> <strong>shows</strong> <strong>that</strong> <strong>Madagascar</strong> <strong>is</strong> <strong>the</strong> <strong>only</strong> <strong>country</strong> <strong>without</strong> <strong>coronavirus</strong> <strong>cases</strong>. ... Explanation: The origin and purpose of <strong>the</strong> <strong>map</strong> <strong>is</strong> unk

In [3]:
exit()

: 